In [1]:
import kagglehub
import csv
import pandas as pd
import math
import numpy as np
import pickle
import os
import getpass

from io import BytesIO
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import create_openai_functions_agent
from langchain.tools import tool
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub # Used to pull predefined prompts from LangChain Hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory  # Used to store chat history in memory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAI

import tiktoken

### Load data and prepare documents

In [2]:
%%time
SCOPES = ['https://www.googleapis.com/auth/drive']

creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

# Authenticate if no valid credentials exist
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    
    # Save the credentials for future use
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# Initialize Google Drive API service
drive_service = build('drive', 'v3', credentials=creds)

# List files in Google Drive
results = drive_service.files().list(
    q="name='train_100.csv'",  # Query to search by filename
    spaces='drive',
    fields="files(id, name)"
).execute()

files = results.get("files", [])

if not files:
    print("No file found.")
else:
    file_id = files[0]['id']
    print(f"File ID: {file_id}")

    # Request to download the file
request = drive_service.files().get_media(fileId=file_id)
file = BytesIO()
downloader = MediaIoBaseDownload(file, request)

done = False
while not done:
    status, done = downloader.next_chunk()
    print(f"Download {int(status.progress() * 100)}%.")

# Move to the beginning of the file
file.seek(0)

# Read CSV into Pandas DataFrame
df = pd.read_csv(file, index_col=0)

# Display the first few rows
df.head()

product_description = []
product_description_len = []

row_num = 0
# Iterating through each row in the dataframe df2
for row in df.iterrows():
    if row_num % 100000 == 0:
        print(f"processing row {row_num}")
    row_num += 1
    product = ""  # Initialize an empty string to accumulate product details

    # Extracting the product title from the current row
    title = row[1]["TITLE"]

    # Checking if the title is valid (not NaN or missing)
    if type(title) != float or not math.isnan(title):
        product += "Title\n" + title + "\n"  # Append the title to the product description

    # Extracting the product description from the current row
    description = row[1]["DESCRIPTION"]

    # Checking if the description is valid (not NaN or missing)
    if type(description) != float or not math.isnan(description):
        product += "Description\n" + description + "\n"  # Append the description to the product details

    # Check if either title or description was added
    added_content = title or description
    if added_content:
        product = product.strip()  # Remove any leading/trailing whitespace
        product_description.append(product)  # Add the formatted product details to the list
        product_description_len.append(len(product))  # Store the length of the product description

print(f"Number of elements {len(product_description)}")
print("Number of items", len(product_description_len))
print("Min length of the description", np.min(product_description_len))
print("Avg length of the description", np.mean(product_description_len))
print("Max length of the description", np.max(product_description_len))

File ID: 1f1E8hjgOWFEXIRsK5gV-j-XD2gYBwPN2
Download 100%.
processing row 0
Number of elements 100
Number of items 100
Min length of the description 18
Avg length of the description 385.9
Max length of the description 1834
CPU times: user 45.7 ms, sys: 12.2 ms, total: 57.8 ms
Wall time: 1.11 s


### Build retrieval chain

In [3]:
%%time
# Chunking the Data(Product Description)
# See this for more details https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.create_documents(product_description)

# Create an embedding model using LangChain.
# See https://python.langchain.com/docs/integrations/text_embedding/ for a list of available embedding models on LangChain
embeddings = OpenAIEmbeddings()

# Create a vector store using the created chuns and the embeddings model
vector = FAISS.from_documents(documents, embeddings)

# Create ChatOpenAI object for acting as an LLM.
# See https://python.langchain.com/docs/integrations/chat/openai/
llm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Create a prompt template that gives the model a persona of a customer
# Create a chain for passing a list of Documents to a model.
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""", output_parser = output_parser)

document_chain = create_stuff_documents_chain(llm, prompt)
# document_chain = prompt | llm

# Creating a retriever
# See https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/vectorstore/
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


CPU times: user 491 ms, sys: 66.2 ms, total: 557 ms
Wall time: 1.63 s


### Setup Tools and agents

In [4]:
@tool
def amazon_product_search(query: str):
    """Search for information about Amazon products.
    For any questions related to Amazon products, this tool must be used."""

    retriever_tool = create_retriever_tool(
        retriever,
        name="amazon_search",
        description="Search for information about Amazon products"
    )
    return retriever_tool.invoke(query)

@tool
def search_tavily(query: str):
    """
    Executes a web search using the TavilySearchResults tool.

    Parameters:
      query (str): The search query entered by the user.

    Returns:
      list: A list of search results containing answers, raw content, and images.
    """
    search_tool = TavilySearchResults(
        max_results=5,
        include_answer=True,  # include direct answers when available
        include_raw_content=True,
        include_images=True
    )

    return search_tool.invoke(query)

tools = [amazon_product_search, search_tavily]

# hwchase17/react is a prompt template designed for ReAct-style
# conversational agents.
prompt = hub.pull("hwchase17/react")

print(prompt.template)

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

react_agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt)

agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)
    

/Users/ericmelz/Data/code/review-genie/venv/lib/python3.13/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [5]:
res = agent_executor.invoke({"input": "What is the best shoes I can find on Amazon?"})



> Entering new AgentExecutor chain...
To find the best shoes available on Amazon, I should search for popular and highly-rated options. I'll use the Amazon product search tool to gather relevant information.  
Action: amazon_product_search  
Action Input: "best shoes"  The Remora Climbing Shoe is Mad Rock's do-it-all slipper for climbers who can't have separate shoes for boulders, sport routes, and gyms. With a moderately stiff, slightly downturned design, the Remora performs on any climb at steep to vertical

steep to vertical angles. Science Friction rubber edges on jibs with ease, and it encapsulates the forefoot for toe-hooking prowess. The SynFlex upper conforms to your foot without much stretch.

Title
adidas Men's Predator 18+ FG Firm Ground Soccer Cleats
Description
adidas Predator 18+ FG- Black 7.5

Title
Kenneth Cole REACTION Men's Crespo Loafer B Shoe, Cognac, 10 M USI need to gather more options to provide a comprehensive answer about the best shoes available on Amazon. I

In [6]:
res = agent_executor.invoke({"input": "What is the current weather in Toronto?"})



> Entering new AgentExecutor chain...
I need to find the current weather information for Toronto. Since this is a general knowledge question, I will perform a web search to get the latest weather updates.  
Action: search_tavily  
Action Input: "current weather in Toronto"  [{'title': 'Weather in Toronto', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Toronto', 'region': 'Ontario', 'country': 'Canada', 'lat': 43.6667, 'lon': -79.4167, 'tz_id': 'America/Toronto', 'localtime_epoch': 1742513841, 'localtime': '2025-03-20 19:37'}, 'current': {'last_updated_epoch': 1742513400, 'last_updated': '2025-03-20 19:30', 'temp_c': 3.0, 'temp_f': 37.4, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 16.1, 'wind_kph': 25.9, 'wind_degree': 313, 'wind_dir': 'NW', 'pressure_mb': 1003.0, 'pressure_in': 29.61, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 75, 'cloud': 100, 'feelslike_c': -2.2

### Explore tiktoken

In [19]:
encoding = tiktoken.encoding_for_model("text-embedding-3-small")
text = "This is a test document."
tokens = encoding.encode(text)
print(f"Number of tokens: {len(tokens)}")
print(f"Tokens: {tokens}")

for i in range(1, len(tokens)+1):
    decoded_text = encoding.decode(tokens[:i])
    print(f"Decoded text[:{i}]: |{decoded_text}|")

Number of tokens: 6
Tokens: [2028, 374, 264, 1296, 2246, 13]
Decoded text[:1]: |This|
Decoded text[:2]: |This is|
Decoded text[:3]: |This is a|
Decoded text[:4]: |This is a test|
Decoded text[:5]: |This is a test document|
Decoded text[:6]: |This is a test document.|


In [20]:
docs = [
    "This is the first document.",
    "This is the second document.",
    "And this is the third one."
]
total_tokens = 0
for doc in docs:
    tokens = encoding.encode(doc)
    total_tokens += len(tokens)
print(f"\nTotal number of tokens across all documents: {total_tokens}")


Total number of tokens across all documents: 19


In [38]:
documents[0].page_content

'Title\nArtzFolio Tulip Flowers Blackout Curtain for Door, Window & Room | Eyelets & Tie Back | Canvas Fabric | Width 4.5feet (54inch) Height 5 feet (60 inch); Set of 2 PCS'

In [39]:
encoding.encode(documents[0].page_content)

[3936,
 198,
 9470,
 89,
 37,
 73267,
 43316,
 575,
 52645,
 5348,
 412,
 84657,
 369,
 25166,
 11,
 13956,
 612,
 10637,
 765,
 28929,
 10145,
 612,
 59825,
 6984,
 765,
 26782,
 37407,
 765,
 25650,
 220,
 19,
 13,
 20,
 1897,
 295,
 320,
 4370,
 37850,
 8,
 22147,
 220,
 20,
 7693,
 320,
 1399,
 17560,
 1237,
 2638,
 315,
 220,
 17,
 92197]

In [40]:
len(encoding.encode(documents[0].page_content))

52

In [42]:
token_counts = []
for doc in documents:
    page_content = doc.page_content
    encoding = tiktoken.encoding_for_model("text-embedding-3-small")
    token_count = len(encoding.encode(page_content))
    token_counts.append(token_count)

In [43]:
print(token_counts)

[52, 25, 25, 62, 22, 25, 79, 50, 40, 15, 52, 56, 48, 46, 29, 11, 30, 68, 56, 59, 63, 60, 67, 56, 20, 36, 48, 47, 36, 11, 24, 29, 47, 49, 49, 14, 13, 25, 25, 15, 36, 36, 67, 58, 54, 46, 23, 13, 10, 19, 62, 13, 16, 20, 57, 63, 67, 68, 22, 24, 87, 67, 38, 42, 52, 45, 10, 35, 49, 46, 60, 69, 74, 73, 47, 44, 50, 44, 51, 29, 11, 22, 30, 56, 70, 53, 67, 48, 48, 44, 22, 15, 53, 17, 47, 28, 16, 6, 28, 47, 49, 64, 8, 8, 25, 52, 32, 46, 40, 62, 64, 62, 78, 69, 11, 16, 8, 25, 19, 11, 28, 33, 20, 44, 42, 22, 18, 67, 50, 83, 112, 4, 54, 60, 70, 53, 57, 7, 11, 45, 51, 51, 51, 4, 14, 37, 12, 33, 63, 65, 62, 62, 79, 24, 19, 56, 42, 51, 37, 28, 22, 54, 53, 40, 16, 43, 46, 22, 57, 52, 66, 53, 57, 45, 25, 50, 53, 47, 22, 10, 25, 43, 46, 45, 44, 44, 48, 48, 24, 17, 11, 21, 51, 51, 32, 12, 27, 24, 57, 101, 103, 101, 103, 56, 8, 19, 35, 11, 28, 44, 54, 65, 54, 44, 53, 75, 22, 67, 81, 63, 11, 44, 60, 62, 76, 47, 34, 29, 6, 23, 54, 58, 24, 13, 41, 38, 15, 70]


In [45]:
total_tokens = sum(token_counts)
print(total_tokens)

9963


In [52]:
tokens_per_page = 800
pages = total_tokens / tokens_per_page
pages_per_dollar = 62500
cost_in_dollars = pages / pages_per_dollar
cost_in_cents = cost_in_dollars * 100
print(f'{cost_in_cents=:.8f}')

cost_in_cents=0.01992600
